In [46]:
import sqlite3
import csv


con = sqlite3.connect("commandes.db3")
cur = con.cursor()

###############                 Creating Tables             #################

queriesCreateTables = [
    '''
    DROP TABLE IF EXISTS customer;
    CREATE TABLE customer(
        customer_id INTEGER NOT NULL PRIMARY KEY,
        country VARCHAR(100) NOT NULL
    );
    ''',
    '''
    DROP TABLE IF EXISTS customer_order;
    CREATE TABLE customer_order(
        order_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
        invoice_nb INTEGER NOT NULL,
        invoice_date DATE,
        customer_id INTEGER NOT NULL,
        FOREIGN KEY (customer_id)
            REFERENCES customer (customer_id)
    );
    ''',
    '''
    DROP TABLE IF EXISTS order_detail;
    CREATE TABLE order_detail(
        order_detail_id INTEGER NOT NULL,
        quantity INTEGER NOT NULL,
        order_id INTEGER,
        product_id INTEGER NOT NULL,
        FOREIGN KEY (product_id)
            REFERENCES product (product_id)
    );
    ''',
    '''
    DROP TABLE IF EXISTS product;
    CREATE TABLE product(
        product_id INTEGER NOT NULL PRIMARY KEY,
        description TEXT,
        price FLOAT
    );
    '''
]

for query in queriesCreateTables:
    try:
        cur.executescript(query)  
        print("Query executed successfully.")
    except sqlite3.Error as e:
        print(f" Error : {e}")


con.commit()
##############################################################################################

Query executed successfully.
Query executed successfully.
Query executed successfully.
Query executed successfully.


In [47]:
def getIds(query, cur):
    return [str(item[0]) for item in cur.execute(query).fetchall()] # to form a list with string elements
                                                                    # fetchall() returns ((1,),(2,),(3,))
                                                                    # we transform into  ['1','2','3']

queriesId = {
    "idProduct": "SELECT product_id FROM product",
    "idCustomer": "SELECT customer_id FROM customer",
    "idOrder": "SELECT order_id FROM customer_order",
    "idOrderDetail": "SELECT order_detail_id FROM order_detail"
}

# Lists of Ids already in database
idProductList = getIds(queriesId['idProduct'], cur)
idCustomerList = getIds(queriesId['idCustomer'], cur)
idOrderList = getIds(queriesId['idOrder'], cur)
idOrderDetailList = getIds(queriesId['idOrderDetail'], cur)

#############           Reading from CSV            #####################

file=open("orders.csv", "r")
reader=csv.reader(file, delimiter=";")
for i, row in enumerate(reader) :    
    if i==0:
        tabTitres=row
        continue
        
    order_id=row[tabTitres.index('order_id')]
    invoice_nb=row[tabTitres.index('invoice_nb')]
    invoice_date=row[tabTitres.index('invoice_date')]
    customer_id=row[tabTitres.index('customer_id')]
    country=row[tabTitres.index('country')]
    order_detail_id=row[tabTitres.index('order_detail_id')]
    quantity=row[tabTitres.index('quantity')]
    product_id=row[tabTitres.index('product_id')]
    description=row[tabTitres.index('description')]
    price=row[tabTitres.index('price')]
    
   
    ##    PRODUCTS 
    if  product_id not in idProductList :
        idProductList.append(product_id)
        insertProduct=f"""INSERT INTO product (product_id, description, price)
        VALUES(?,?,?);"""   
        cur.execute(insertProduct, (product_id,description, price))
        print("Row inserted in Products")
    else:
        print("Product already exists in table")

    ##    CUSTOMER 
    if customer_id not in idCustomerList :
        idCustomerList.append(customer_id)
        insertCustomer=f"""INSERT INTO customer (customer_id, country)
        VALUES(?,?);"""   
        cur.execute(insertCustomer, (customer_id,country))
        print("Row inserted in Customers")
    else:
        print("Customer already exists in table")


    ##    ORDER 
    if order_id not in idOrderList :
        idOrderList.append(order_id)
        insertorder=f"""INSERT INTO customer_order (invoice_nb, invoice_date, customer_id)
        VALUES(?,?,?);"""   
        cur.execute(insertorder, (invoice_nb,invoice_date, customer_id))
        print("Row inserted in Customer Order")
    else:
        print("Order already exists in table")

    ##    ORDER DETAIL 
    if order_detail_id not in idOrderDetailList :
        idOrderDetailList.append(order_detail_id)
        insertOrderDetail=f"""INSERT INTO order_detail (order_detail_id, quantity, order_id,product_id)
        VALUES(?,?,?,?);"""   
        cur.execute(insertOrderDetail, (order_detail_id,quantity,order_id, product_id))
        print("Row inserted in Orders")
    else:
        print("Order detail already exists in table")   

    
    con.commit()
    if i>=10 :
        break


Row inserted in Products
Row inserted in Customers
Row inserted in Customer Order
Row inserted in Orders
Row inserted in Products
Customer already exists in table
Order already exists in table
Row inserted in Orders
Row inserted in Products
Customer already exists in table
Order already exists in table
Row inserted in Orders
Row inserted in Products
Customer already exists in table
Order already exists in table
Row inserted in Orders
Row inserted in Products
Customer already exists in table
Order already exists in table
Row inserted in Orders
Row inserted in Products
Customer already exists in table
Order already exists in table
Row inserted in Orders
Row inserted in Products
Customer already exists in table
Order already exists in table
Row inserted in Orders
Row inserted in Products
Customer already exists in table
Row inserted in Customer Order
Row inserted in Orders
Row inserted in Products
Customer already exists in table
Order already exists in table
Row inserted in Orders
Row in

In [29]:
print(idProductList)
print(idCustomerList)
print(idOrderList)
print(idOrderDetailList)

['756', '772', '773', '784', '1192', '1480', '1481', '1490', '1491', '1599', '1602', '1603', '1606', '2629', '2650', '2803', '2804', '2856', '3070', '3250']
['541', '4049']
['1', '2', '3']
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']


In [48]:
querySelectJoin='''SELECT DISTINCT order_detail.order_detail_id, customer_order.order_id, customer_order.invoice_nb,
            customer_order.invoice_date, customer.customer_id, customer.country,
            order_detail.quantity, 
            product.product_id, product.description, product.price
        FROM customer
            INNER JOIN customer_order ON customer.customer_id = customer_order.customer_id
            INNER JOIN order_detail ON customer_order.order_id = order_detail.order_id
            INNER JOIN product ON order_detail.product_id = product.product_id
         LIMIT 10'''
data = cur.execute(querySelectJoin)
print(data.fetchall())

[(1, 1, 536365, '2010/12/01', 4049, 'United Kingdom', 6, 756, 'GLASS STAR FROSTED T-LIGHT HOLDER', 4.25), (2, 1, 536365, '2010/12/01', 4049, 'United Kingdom', 2, 1606, 'SET 7 BABUSHKA NESTING BOXES', 7.65), (3, 1, 536365, '2010/12/01', 4049, 'United Kingdom', 6, 2650, 'WHITE METAL LANTERN', 3.39), (4, 1, 536365, '2010/12/01', 4049, 'United Kingdom', 6, 2803, 'RED WOOLLY HOTTIE WHITE HEART.', 3.39), (5, 1, 536365, '2010/12/01', 4049, 'United Kingdom', 6, 2804, 'KNITTED UNION FLAG HOT WATER BOTTLE', 3.39), (6, 1, 536365, '2010/12/01', 4049, 'United Kingdom', 8, 2856, 'CREAM CUPID HEARTS COAT HANGER', 2.75), (7, 1, 536365, '2010/12/01', 4049, 'United Kingdom', 6, 3250, 'WHITE HANGING HEART T-LIGHT HOLDER', 2.55), (8, 2, 536366, '2010/12/01', 4049, 'United Kingdom', 6, 1490, 'HAND WARMER RED POLKA DOT', 1.85), (9, 2, 536366, '2010/12/01', 4049, 'United Kingdom', 6, 1491, 'HAND WARMER UNION JACK', 1.85), (10, 3, 536367, '2010/12/01', 541, 'United Kingdom', 3, 772, 'HOME BUILDING BLOCK WORD'

In [ ]:
cur.close()
con.close()